In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import unittest
import pandas as pd

import os
import inspect
import sys
from pandas._testing import assert_frame_equal


sys.path.insert(1, '../src/preprocessing/')
from preprocessing_lib import nn_format


In [38]:
class TestCreateTimeseries(unittest.TestCase):
    
    def test_nn_format(self):
        df = pd.DataFrame({
            "start_time": ["2018-01-01 00:00:00"] * 3 + ["2018-01-01 01:00:00"] * 3 +  ["2018-01-01 02:00:00"] * 3,
            "from_station_id": [1, 2, 3, 1, 2, 3, 1, 2, 3],
            "quantity": [100,200,300,101,201,301, 102, 202, 302]
        })
        df = df.reset_index(drop=True).set_index("start_time")

        expected_df = pd.DataFrame({ 
            "start_time": ["2018-01-01 00:00:00", "2018-01-01 01:00:00", "2018-01-01 02:00:00"],
            "quantity_1": [100, 101, 102],
            "quantity_2": [200, 201, 202],
            "quantity_3": [300, 301, 302],
        })
        expected_df = expected_df.reset_index(drop=True).set_index("start_time")

        df_nn = nn_format(df)
        assert_frame_equal(expected_df, df_nn)

        

In [39]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.009s

OK
